In [ ]:
from bluesky import RunEngine
from bluesky.plans import scan
from ophyd.sim import motor, det
from bluesky_widgets.utils.streaming import stream_documents_into_runs
from bluesky_widgets.models.plot_builders import PromptPlotter
from bluesky_widgets.jupyter.figures import JupyterFigures

from bluesky_widgets.models.plot_specs import LineSpec, AxesSpec, FigureSpec


def prompt_line_builder(run):                                                   
    """                                                                         
    This is a simple example.                                                   

    This makes a hard-coded assumption that the data has columns "motor" and    
    "det" in the primary stream.                                                
    """                                                                         

    def func(run):                                                              
        "Return any arrays x, y. They must be of equal length."                 
        # *Lazily* read the data so that large arrays are not loaded unless     
        # the yare used.                                                        
        ds = run.primary.read()                                                 
        # Do any computation you want in here....                               
        return ds["motor"], ds["det"]                                           

    label = f"Scan {run.metadata['start']['scan_id']}"                          
    line_spec = LineSpec(func, run, label=label)                           
    axes_spec = AxesSpec(lines=[line_spec], x_label="motor", y_label="det")     
    figure_spec = FigureSpec((axes_spec,), title="det v motor")                 

    return figure_spec

RE = RunEngine()
model = PromptPlotter([prompt_line_builder])
view = JupyterFigures(model.figures)
RE.subscribe(stream_documents_into_runs(model.add_run))

In [ ]:
view

In [ ]:
model.figures.clear()

In [ ]:
motor.delay = 0.1  # Turn up simulated motor movement delay.

In [ ]:
def plan():
    for i in range(1, 5):
        yield from scan([det], motor, -1, 1, 1 + 2 * i)

In [ ]:
RE(plan())

In [ ]:
del model.figures[2]

In [ ]:
model.figures[0].axes[0].by_label["Scan 1"][0].style.update(color="red")

In [ ]:
# Generate example data. (This takes a couple seconds to simulate some scans.)
from bluesky_widgets.examples.utils.generate_msgpack_data import get_catalog
catalog = get_catalog()
scans = catalog.search({"plan_name": "scan"})

In [ ]:
model.add_run(scans[-2])